In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset download


In [ ]:
#!pip install aicrowd-cli

In [ ]:
import os
# os.getcwd()
os.chdir('/content/drive/MyDrive/dataset_cv_Assignment5')
os.getcwd()
!unzip -q /content/drive/MyDrive/dataset_cv_Assignment5/train_images.zip 

replace train_images/815c3e59f4.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
os.getcwd()
!unzip -q /content/drive/MyDrive/dataset_cv_Assignment5/test_images.zip 

# Dataset handling

In [ ]:
mport pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
PATH_TRAINING='/content/drive/MyDrive/dataset_cv_Assignment5/train_images'
PATH_TESTING='/content/drive/MyDrive/dataset_cv_Assignment5/test_images'
TRAIN_CLASS_PATH='/content/drive/MyDrive/dataset_cv_Assignment5/train.csv'
TEST_CLASS_PATH='/content/drive/MyDrive/dataset_cv_Assignment5/test.csv'
#NUM_TRAINING=len(os.listdir(PATH_TRAINING))
#NUM_TESTING=len(os.listdir(PATH_TESTING))
SUBSET_TRAINING=1000
SUBSET_TESTING=500

In [ ]:
class dataprep1(Dataset):
    def __init__(self,data_list,data_dir = './',transform=None,train=True):
        super().__init__()
        self.data_list = data_list
        self.data_dir = data_dir
        self.transform = transform
        self.train = train
    def __getitem__(self,item):
        if self.train:
            img_name,label = self.data_list.iloc[item]
        else:
            img_name = self.data_list.iloc[item]['ImageId']
        img_path = os.path.join(self.data_dir,img_name)
        img1 = cv2.imread(img_path,1)
        img1 = cv2.resize(img,(256,256))
        if self.transform is not None:
            img1 = self.transform(img1)
        if self.train:
            return {
              'gt' : img1,
              'label' : torch.tensor(label)
          }
        else:
            return {
              'gt':img1
          }
          
    def __len__(self):
        return self.data_list.shape[0]
    
    

In [ ]:

batch = 64

train = pd.read_csv(TRAIN_LABELS)

num = train['ClassName'].value_counts()
classes = train['ClassName'].unique()
num_classes = len(classes)

label_enc = preprocessing.LabelEncoder()
targets = label_enc.fit_transform(train['ClassName'])
ntrain = train
ntrain['ClassName'] = targets

train_trsf = transforms.Compose([
    transforms.ToPILImage(),
    
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))
])

data_train= dataprep1(data_list= ntrain,data_dir = TRAIN_PATH,transform = train_trsf)

valid_size = 0.2
num = train_data.__len__()
# Dividing the indices for train and cross validation
indices = list(range(num))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num))
train_idx,valid_idx = indices[split:], indices[:split]

#Create Samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

loadtrainer = DataLoader(data_train, batch_size = batch, sampler = train_sampler, num_workers=20)
loadvalid = DataLoader(data_train, batch_size = batch, sampler = valid_sampler, num_workers=20)

transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5) , (0.5,0.5,0.5))
])
test_path = PATH_TESTING
test = pd.read_csv(TEST_LABELS)
test_data = data_handler(data_list= test,data_dir = test_path,transform = transforms_test,train=False)

testloader = DataLoader(test_data, batch_size=batch, shuffle=False, num_workers=30)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class conv1(nn.Module):
    def __init__(self,channels_list,lkernels,strides_list,padding_list,pool_type = 'maxpool',bn = True, activation = 'relu'):
        super(conv1,self).__init__()
        
        num_layers = len(channels_list) - 1
        if(pool_type == 'maxpool'):   self.pool    = nn.MaxPool2d(2,2)
        elif(pool_type == 'avgpool'): self.pool    = nn.AvgPool2d(2,2)
        elif(pool_type == 'l1pool'):  self.pool    = nn.LPPool2d(1, 2, stride = 2)
        elif(pool_type == 'l2pool'):  self.pool    = nn.LPPool2d(2, 2, stride = 2)
        else: print('pool_type not supported')
        self.convs = []
        for i in range(num_layers):
            tmp = [basic_block(channels_list[i],channels_list[i+1], kernel_size = lkernels[i], stride = strides_list[i], padding = padding_list[i], bn = bn, activation = activation),self.pool]
            for layer in tmp:
                self.convs.append(layer)
        self.convs = nn.ModuleList(self.convs)
    
    def forward(self,x):
        for layer in self.convs:
            x = layer(x)
        return x

class basic_block(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size = 3,stride = 1,padding = 0,bn = True, activation = 'relu'):
        super(basic_block,self).__init__()
        
        self.bn        = bn 
        self.conv      = nn.Conv2d(in_channels=in_channels,kernel_size=kernel_size,out_channels=out_channels,stride=stride,padding=padding)
        self.batchnorm = nn.BatchNorm2d(num_features=out_channels)
        if(activation == 'relu'):
            self.relu  = nn.ReLU()
        elif(activation == 'elu'):
            self.relu  = nn.ELU()
        elif(activation == 'leaky_relu'):
            self.relu  = nn.LeakyReLU(0.1)
        else:
            print('activation not supported')

    def forward(self,x):
        output = self.conv(x)
        if(self.bn):
            output = self.batchnorm(output)
        output = self.relu(output)
        return output

def calc_ln_in(inp_size,out_channel,lkernels,strides_list,padding_list,pooling_kernel,pooling_stride):
    for i in range(len(lkernels)):
        inp_size = 1 + ((inp_size - lkernels[i] + 2*padding_list[i])//strides_list[i])
        inp_size = 1 + ((inp_size - pooling_kernel)//pooling_stride)
    if(inp_size < 1):
        print('Error: Incorrect nn.linear input. Got in_features = 1. Adjust parameters list')
    else:
        return out_channel*inp_size**2
    
class ln_list(nn.Module):
    def __init__(self,lin_list):
        super(ln_list,self).__init__()
        num_layers = len(lin_list) - 1
        self.linears = []
        for i in range(num_layers):
            tmp = [nn.Linear(lin_list[i],lin_list[i+1]),nn.ReLU()]
            if(i == num_layers - 1):
                self.linears.append(nn.Linear(lin_list[i],lin_list[i+1]))
                break
            for layer in tmp:
                self.linears.append(layer)
        self.linears = nn.ModuleList(self.linears)

    def forward(self,x):
        for layer in self.linears:
            x = layer(x)
        return x

   
class DNN(nn.Module):
    def __init__(self,in_size = 32,channels_list = [3,16,32],lkernels = [3,5],strides_list = 1,padding_list = [1,0],lin_list = [512,100],pool_type = 'maxpool',bn = False, dropout = False, activation = 'relu'):
        super(CNN, self).__init__()
        self.pool_type = pool_type
        self.d = dropout
        if(dropout):
            self.dropout   = nn.Dropout2d(p = 0.2)
        
        num_layers = len(channels_list) - 1
        if(type(lkernels) == int): lkernels = [lkernels for _ in range(num_layers)]
        if(type(strides_list) == int): strides_list = [strides_list for _ in range(num_layers)]
        if(type(padding_list) == int): padding_list = [padding_list for _ in range(num_layers)]
        
        self.in_features = calc_ln_in(in_size,out_channel = channels_list[-1],lkernels = lkernels,strides_list = strides_list,padding_list = padding_list,pooling_kernel = 2,pooling_stride = 2)
        self.convs = conv1(channels_list, lkernels = lkernels, strides_list = strides_list,padding_list = padding_list, pool_type = self.pool_type, bn = bn, activation = activation)
        self.linears = ln_list([self.in_features,*lin_list])
        
    def forward(self, x):
        x = self.convs(x)
        if(self.d):
            x = self.dropout(x)
        x = x.view(-1,self.in_features)
        x = self.linears(x)
        return x

In [ ]:
class loss1(nn.Module):
    def forward(self,outputs,labels):
        if(self.loss_type == 'l1' or self.loss_type == 'l2'):
            onh = torch.FloatTensor(len(labels),self.num_classes).to(device)
            onh.zero_()
            onh.scatter_(1,labels.view(len(labels),1),1)
            arr = onh - F.softmax(outputs,dim=1)
            return torch.sum(torch.abs(arr).pow(self.p)/len(labels))
        elif(self.loss_type == 'cross_entropy'):
            return self.criterion(outputs,labels)
    
    def __init__(self,loss_type = 'l2', num_classes=100):
        super(loss1,self).__init__()
        self.loss_type = loss_type
        self.num_classes = num_classes
        if(self.loss_type == 'l1'):
            self.p = 1
        elif(self.loss_type == 'l2'):
            self.p = 2
        elif(self.loss_type == 'cross_entropy'):
            self.criterion = nn.CrossEntropyLoss()
          
    

In [ ]:
def acc(labels,out):
    pred = np.argmax(out.detach().cpu().numpy(),axis=1)
    return 100*accuracy_score(labels.cpu().numpy(),pred)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
def train(in_size = 32,channels_list = [3,16,32],lkernels = [3,5],strides_list = 1,padding_list = [1,0],lin_list = [512,100],pool_type = 'maxpool',bn = False, dropout = False,activation = 'relu',loss_type = 'cross_entropy',num_epochs = 10):
    model = DNN(in_size = in_size,channels_list = channels_list,lkernels = lkernels,strides_list = strides_list,padding_list = padding_list,lin_list = lin_list,pool_type = pool_type,bn = bn, dropout = dropout, activation=activation).to(device)
    model = nn.DataParallel(model)
    criterion = loss1(loss_type=loss_type, num_classes=num_classes)
    optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=0.0001)
    train_acc,test_acc = [],[]
    train_loss,test_loss = [],[]
    for epoch in range(num_epochs): 

        running_loss = 0.0
        acc_list = []
        model.train()
        for i, data in enumerate(loadtrainer, 0):
            inputs, labels = data['gt'].squeeze(0).to(device), data['label'].long().to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            acc_list.append(acc(labels,outputs))

        print('Train : [%d] loss: %.3f Accuracy: %.2f' %(epoch , running_loss / i, np.mean(np.array(acc_list))))

        train_loss.append(running_loss/i)
        train_acc.append(np.mean(np.array(acc_list)))
        model.eval()
        running_loss = 0.0
        acc_list = []
        with torch.no_grad():
            for i, data in enumerate(loadvalid, 0):
                inputs, labels = data['gt'].squeeze(0).to(device), data['label'].long().to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item()
                acc_list.append(acc(labels,outputs))

        print('Val : [%d] loss: %.3f Accuracy: %.2f' %(epoch , running_loss / i, np.mean(np.array(acc_list))))
        test_loss.append(running_loss/i)
        test_acc.append(np.mean(np.array(acc_list)))
    

    model.eval()

    preds = []
    with torch.no_grad():
        for images in testloader:
            data = images['gt'].squeeze(0).to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            pr = predicted.detach().cpu().numpy()
            for i in pr:
                preds.append(i)
    
    df = pd.DataFrame(label_enc.inverse_transform(preds),columns=['ClassName'])
    bn_str = 'bn' if bn else ''
    dropout_str = 'dropout' if dropout else ''
    output_str = 'submissionKK_' + '_'.join([loss_type,bn_str,pool_type]) + '.csv'
    df.to_csv(output_str,index=False)

    return model,train_loss,test_loss,train_acc,test_acc

In [ ]:
def plot(train_loss,test_loss,train_acc,test_acc,num_epochs,title = ''):
    x = np.arange(num_epochs)
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(x, train_loss,'r',label='train_loss')
    ax.plot(x, test_loss,'b',label='val_loss')
    plt.xticks(x[::2])
    plt.title('Loss curve' + str(' ') + title)
    ax.legend()
    plt.show()
    fig1 = plt.figure()
    ax = plt.subplot(111)
    ax.plot(x, train_acc,'r',label='train_acc')
    ax.plot(x, test_acc,'b',label='val_acc')
    plt.xticks(x[::2])
    plt.title('Accuracy curve' + str(' ') + title)
    ax.legend()
    plt.show()

In [ ]:
in_size = 256
padding_list = [1,0,0]
pool_type = 'maxpool'
channels_list = [3,6,32,64]
lkernels = [3,5,3]
strides_list = 1
dropout = True
num_epochs = 20
lin_list = [512,num_classes]
bn = True
activation = 'relu'
loss_type = 'cross_entropy'

In [ ]:
_,train_loss,test_loss,train_acc,test_acc = train(in_size,channels_list,lkernels,strides_list,padding_list,lin_list,pool_type,bn,dropout,activation,loss_type,num_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train : [0] loss: 9.383 Accuracy: 8.02
Val : [0] loss: 4.033 Accuracy: 13.40
Train : [1] loss: 3.661 Accuracy: 15.55
Val : [1] loss: 3.755 Accuracy: 17.13
Train : [2] loss: 3.425 Accuracy: 19.64
Val : [2] loss: 3.615 Accuracy: 20.35
Train : [3] loss: 3.238 Accuracy: 22.68
Val : [3] loss: 3.524 Accuracy: 21.28
Train : [4] loss: 3.083 Accuracy: 24.68
Val : [4] loss: 3.434 Accuracy: 21.50
Train : [5] loss: 2.948 Accuracy: 26.55
Val : [5] loss: 3.413 Accuracy: 22.89
Train : [6] loss: 2.833 Accuracy: 29.04
Val : [6] loss: 3.445 Accuracy: 21.39
Train : [7] loss: 2.689 Accuracy: 31.35
Val : [7] loss: 3.404 Accuracy: 22.10
Train : [8] loss: 2.584 Accuracy: 32.52
Val : [8] loss: 3.391 Accuracy: 22.64
Train : [9] loss: 2.474 Accuracy: 35.05
Val : [9] loss: 3.504 Accuracy: 22.30
Train : [10] loss: 2.326 Accuracy: 37.46
Val : [10] loss: 3.476 Accuracy: 22.23
Train : [11] loss: 2.178 Accuracy: 40.43
Val : [11] loss: 3.557 Accuracy: 22.82
Train : [12] loss: 2.023 Accuracy: 44.71
Val : [12] loss: 3.5

Convoltion network with 4 convolution layers and fully connected dense layer for classificaiton used. This modelis trained with batch size of 64 and 20 epochs. Observed training loss is reducing but, validation loss is not reducing in a similar manner. Final validation accuracy of 20% is achieved. 